# Building custom sources using SQL Databases [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dlt-hub/dlt/blob/master/docs/education/dlt-advanced-course/lesson_2_custom_sources_sql_databases_.ipynb) [![GitHub badge](https://img.shields.io/badge/github-view_source-2b3137?logo=github)](https://github.com/dlt-hub/dlt/blob/master/docs/education/dlt-advanced-course/lesson_2_custom_sources_sql_databases_.ipynb)

This lesson covers building flexible and powerful custom sources using the `sql_database` verified source.


![Lesson_2_Custom_sources_SQL_Databases_img1](https://storage.googleapis.com/dlt-blog-images/dlt-advanced-course/Lesson_2_Custom_sources_SQL_Databases_img1.png)


## What you will learn

- How to build a custom pipeline using SQL sources
- How to use `query_adapter_callback`, `table_adapter_callback`, and `type_adapter_callback`
- How to load only new data with incremental loading


Setup & install dlt:

In [1]:
%%capture
!pip install pymysql duckdb dlt

## Step 1: Load data from SQL Databases

We’ll use the [Rfam MySQL public DB](https://docs.rfam.org/en/latest/database.html) and load it into DuckDB:

In [2]:
from typing import Any
from dlt.sources.sql_database import sql_database
import dlt

source = sql_database(
    "mysql+pymysql://rfamro@mysql-rfam-public.ebi.ac.uk:4497/Rfam",
    table_names=["family"],
)

pipeline = dlt.pipeline(
    pipeline_name = "sql_database_example",
    destination = "duckdb",
    dataset_name = "sql_data",
    dev_mode = True,
)

load_info = pipeline.run(source)
print(load_info)

Pipeline sql_database_example load step completed in 2.76 seconds
1 load package(s) were loaded to destination duckdb and into dataset sql_data_20251205123827
The duckdb destination used duckdb:////content/sql_database_example.duckdb location to store data
Load package 1764938307.3676305 is LOADED and contains no failed jobs


Explore the `family` table:

In [3]:
pipeline.dataset().family.df().head()

,rfam_acc,rfam_id,auto_wiki,description,author,seed_source,gathering_cutoff,trusted_cutoff,noise_cutoff,comment,...,ecmli_cal_hits,maxl,clen,match_pair_node,hmm_tau,hmm_lambda,created,updated,_dlt_load_id,_dlt_id
0,RF00001,5S_rRNA,1302,5S ribosomal RNA,"Griffiths-Jones SR, Mifsud W, Gardner PP","Szymanski et al, 5S ribosomal database, PMID:1...",38.0,38.0,37.9,5S ribosomal RNA (5S rRNA) is a component of t...,...,225645,196,120,1,-3.7679,0.71820,2013-10-03 20:41:44+00:00,2024-09-09 21:15:12,1764938307.3676305,pole09o4HaYMTQ
1,RF00002,5_8S_rRNA,1303,5.8S ribosomal RNA,"Griffiths-Jones SR, Mifsud W","Wuyts et al, European LSU rRNA database, PMID:...",42.0,42.0,41.9,5.8S ribosomal RNA (5.8S rRNA) is a component ...,...,407997,265,154,1,-3.4977,0.71788,2013-10-03 20:47:00+00:00,2024-09-10 04:51:11,1764938307.3676305,3TS7nrZdPbbziw
2,RF00003,U1,1304,U1 spliceosomal RNA,"Griffiths-Jones SR, Mifsud W, Moxon SJ, Ontive...","Zwieb C, The uRNA database, PMID:9016512",66.0,66.0,65.9,U1 is a small nuclear RNA (snRNA) component of...,...,410919,262,166,1,-3.7311,0.71618,2013-10-03 20:57:11+00:00,2024-09-09 21:15:12,1764938307.3676305,q684g6AgxSGbCg
3,RF00004,U2,1305,U2 spliceosomal RNA,"Griffiths-Jones SR, Mifsud W, Gardner PP","The uRNA database, PMID:9016512; Griffiths-Jon...",46.0,46.0,45.9,U2 is a small nuclear RNA (snRNA) component of...,...,404187,292,192,1,-3.4862,0.71304,2013-10-03 20:58:30+00:00,2024-09-10 04:51:11,1764938307.3676305,9/H+ItOMdk55YA
4,RF00005,tRNA,1306,tRNA,"Eddy SR, Griffiths-Jones SR, Mifsud W",Eddy SR,29.0,29.0,28.9,Transfer RNA (tRNA) molecules are approximatel...,...,281040,217,71,1,-2.6284,0.73562,2013-10-03 21:00:26+00:00,2024-09-10 04:51:11,1764938307.3676305,TtBWbmEFUeWgtg


## Step 2: Customize SQL queries with `query_adapter_callback`

You can fully rewrite or modify the SQL SELECT statement per table.


### Filter rows using a WHERE clause

In [5]:
from sqlalchemy import text
from dlt.sources.sql_database.helpers import SelectClause, Table
import os

### Use sql_database without always declaring the url

def query_adapter_callback(query: SelectClause, table: Table) -> SelectClause:
  return text(f"SELECT * FROM {table.fullname} WHERE rfam_id like '%bacteria%'")

In [ ]:
from sqlalchemy import text
from dlt.sources.sql_database.helpers import SelectClause, Table


def query_adapter_callback(query: SelectClause, table: Table) -> SelectClause:
    return text(f"SELECT * FROM {table.fullname} WHERE rfam_id like '%bacteria%'")

To be able to use `sql_database` and not have to declare the connection string each time, we save it as an environment variable. This can also (should preferably) be done in `secrets.toml`

In [ ]:
import os

os.environ[
    "SOURCES__SQL_DATABASE__CREDENTIALS"
] = "mysql+pymysql://rfamro@mysql-rfam-public.ebi.ac.uk:4497/Rfam"

In [ ]:
filtered_resource = sql_database(
    query_adapter_callback=query_adapter_callback, table_names=["family"]
)

Let's save this filtered data:

In [ ]:
info = pipeline.run(filtered_resource, table_name="bacterias")
print(info)

Explore the data:

In [ ]:
pipeline.dataset().bacterias.df().head()

### **Question 1**:

How many rows are present in the `bacterias` table?

>Answer this question and select the correct option in the homework Quiz.


## Step 3: Modify table schema with `table_adapter_callback`

Add columns, change types, or transform schema using this hook.


### Example: Add computed column `max_timestamp`

In [ ]:
import sqlalchemy as sa


def add_max_timestamp(table: Table) -> Any:
    max_ts = sa.func.greatest(table.c.created, table.c.updated).label("max_timestamp")
    subq = sa.select(*table.c, max_ts).subquery()
    return subq

Use it with `sql_table`:

In [ ]:
from dlt.sources.sql_database import sql_table

table = sql_table(
    table="family",
    table_adapter_callback=add_max_timestamp,
    incremental=dlt.sources.incremental("max_timestamp"),
)

info = pipeline.run(table, table_name="family_with_max_timestamp")
print(info)

Let's check out if this column exists!

In [ ]:
pipeline.dataset().family_with_max_timestamp.df().head()

## Step 4: Adapt column data types with `type_adapter_callback`

When the default types don’t match what you want in the destination, you can remap them.

Let's look at the schema that has already been loaded:

In [ ]:
schema = pipeline.default_schema.to_dict()["tables"]["family"]["columns"]
for column in schema:
    print(schema[column]["name"], ":", schema[column]["data_type"])

Lets change `hmm_lambda` from decimal to float.

💡 Quick fyi: The `float` data type is:
- Fast and uses less space
- But it's approximate — you may get 0.30000000000000004 instead of 0.3
- Bad for money, great for probabilities, large numeric ranges, scientific values

### Example: Change data types

In [ ]:
import sqlalchemy as sa
from sqlalchemy.types import Float


def type_adapter_callback(sql_type: Any) -> Any:
    if isinstance(sql_type, sa.Numeric):
        return Float
    return sql_type

Use it with `sql_database`:

In [ ]:
new_source = sql_database(
    type_adapter_callback=type_adapter_callback, table_names=["family"]
)

info = pipeline.run(new_source, table_name="type_changed_family")
print(info)

👀 Can you see how the column data types have changed?

In [ ]:
schema1 = pipeline.default_schema.to_dict()["tables"]["family"]["columns"]
schema2 = pipeline.default_schema.to_dict()["tables"]["type_changed_family"]["columns"]
column = "trusted_cutoff"

print("For table 'family':", schema1[column]["name"], ":", schema1[column]["data_type"])
print(
    "For table 'type_changed_family':",
    schema2[column]["name"],
    ":",
    schema2[column]["data_type"],
)

### **Question 2**:

How many columns had their type changed in the `type_changed_family` table?


## Step 5: Incremental loads with `sql_database`
Track only new rows using a timestamp or ID column.

We'll also be looking at where these incremental values are stored.

Hint: they are stored in [dlt state](https://dlthub.com/docs/general-usage/state).

In [ ]:
import json

with open(
    "/var/dlt/pipelines/sql_database_example/state.json", "r", encoding="utf-8"
) as f:
    data = json.load(f)

data["sources"]["sql_database"]["resources"]["family"]["incremental"].keys()

In [ ]:
from dlt.sources.sql_database import sql_database
import pendulum

source = sql_database(table_names=["family"])
source.family.apply_hints(
    incremental=dlt.sources.incremental(
        "updated", initial_value=pendulum.datetime(2024, 1, 1)
    )
)

info = pipeline.run(source)
print(info)

In [ ]:
import json

with open(
    "/var/dlt/pipelines/sql_database_example/state.json", "r", encoding="utf-8"
) as f:
    data = json.load(f)

data["sources"]["sql_database"]["resources"]["family"]["incremental"].keys()

## **Rename tables for `sql_database` source**



In [ ]:
import dlt
from dlt.sources.sql_database import sql_database

source = sql_database(table_names=["family"])

# Loop through each resource (table) in the source
for _resource_name, resource in source.resources.items():
    # Rename the target table by prefixing with "xxxx__"
    resource.apply_hints(table_name=f"xxxx__{resource.name}")


pipeline = dlt.pipeline(
    pipeline_name="sql_db_prefixed_tables",
    destination="duckdb",
    dataset_name="renamed_tables",
)


print(pipeline.run(source))
pipeline.dataset().row_counts().df()

✅ ▶ Proceed to the [next lesson](https://colab.research.google.com/drive/1P8pOw9C6J9555o2jhZydESVuVb-3z__y#forceEdit=true&sandboxMode=true)!